## COMP 6934 | Project 02

**Team Members:**<br>
Mehadi Hassan (202287115) <br>
Mohammad Shehabul Islam (202196528) <br>
**Submission Date**: April 05, 2024


In [1]:
import pandas as pd
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [2]:
df_stroke = pd.read_csv(
    "stroke.csv",
    parse_dates=[
        "Emergency Dept Time",
        "Admission Time",
        "Discharge Time",
        "CT Scan Time",
        "TPA Time",
        "ICU Arrival Time",
        "ICU Checkout Time",
        "Neurology Ward Arrival Time",
        "Occupational Therapist Visit",
        "Speech Pathologist Visit",
        "Physiotherapist Visit",
        "Dietitian Visit",
        "Social Worker Visit",
        "Cardiologist Visit",
        "Neurologist Visit",
    ],
)

In [3]:
time_list = {
    "Less than 5 Min": pd.Timedelta(days=0, minutes=5),
    "Less than 15 Min": pd.Timedelta(days=0, minutes=15),
    "Less than 30 Min": pd.Timedelta(days=0, minutes=30),
    "Less than 1 Hour": pd.Timedelta(days=0, hours=1, minutes=0),
    "Less than 3 Hour": pd.Timedelta(days=0, hours=3, minutes=0),
    "Less than 6 Hour": pd.Timedelta(days=0, hours=6, minutes=0),
    "Less than 12 Hour": pd.Timedelta(days=0, hours=12, minutes=0),
    "Less than 24 Hour": pd.Timedelta(days=0, hours=24, minutes=0),
    "Less than 48 Hour": pd.Timedelta(days=0, hours=48, minutes=0),
    "More than 48 Hour": pd.Timedelta(days=0, hours=48, minutes=0),
}

In [4]:
def get_count(df, time_range):
    if "Less" in time_range:
        df = df[(df < time_list[time_range])]
    else:
        df = df[(df > time_list[time_range])]

    return df.count()

In [5]:
def emergency_to_other_dept_time(
    time_range,
    admission_time,
    ct_scan_time,
    tpa_time,
    neurologist_visit_time,
    neurologist_ward_time,
    comorbidity,
    gender,
):

    df = df_stroke.copy()
    plot_df = pd.DataFrame()
    count_values = {}

    if comorbidity == "Yes":
        df = df[df["Comorbidities"] == True]
    elif comorbidity == "No":
        df = df[df["Comorbidities"] == False]

    if gender == "Male":
        df = df[df["Gender"] == "M"]
    elif gender == "Female":
        df = df[df["Gender"] == "F"]

    if admission_time:
        plot_df["mean_admission_time"] = (
            df["Admission Time"] - df["Emergency Dept Time"]
        )
        count_values["Admitted to Hospital"] = get_count(
            plot_df["mean_admission_time"], time_range
        )

    if ct_scan_time:
        plot_df["mean_ct_scan_time"] = df["CT Scan Time"] - df["Emergency Dept Time"]
        count_values["Performed CT Scan"] = get_count(
            plot_df["mean_ct_scan_time"], time_range
        )

    if tpa_time:
        rows_with_tpa_time = df[df['TPA Time'].notnull()]
        plot_df["mean_tpa_time"] = rows_with_tpa_time["TPA Time"] - rows_with_tpa_time["Emergency Dept Time"]
        count_values[f"Performed tPA (Total Patients undergone TPA: {rows_with_tpa_time['TPA Time'].count()})"] = get_count(
            plot_df["mean_tpa_time"], time_range
        )

    if neurologist_ward_time:
        plot_df["mean_neurologist_ward_time"] = (
            df["Neurology Ward Arrival Time"] - df["Emergency Dept Time"]
        )
        count_values["Arrived at Neurology Ward"] = get_count(
            plot_df["mean_neurologist_ward_time"], time_range
        )

    if neurologist_visit_time:
        plot_df["mean_neurologist_visit_time"] = (
            df["Neurologist Visit"] - df["Emergency Dept Time"]
        )
        count_values["Neurologist Visited the Patient"] = get_count(
            plot_df["mean_neurologist_visit_time"], time_range
        )

    labels = list(count_values.keys())[::-1]
    values = list(count_values.values())[::-1]

    fig = go.Figure(go.Bar(
        y=labels,
        x=values,
        orientation='h',
        marker=dict(color=values),
        text=values,
        textposition='auto'
    ))

    fig.update_layout(
        xaxis_title=f'Number of Patients (Total: {df_stroke["Age"].count()})',
        yaxis=dict(tickfont=dict(size=12))
    )

    return fig

In [6]:
def extra_dept_care(categories, gender, comorbidity):
    df = df_stroke.copy()
    counts = []
    parents = []
    labels = []
    for column in categories:

        parents.append("")
        labels.append(column)
        percentage = df_stroke[column].count()
        counts.append(int(round(percentage, 0)))

        if gender == 'All':
            male_percentage = df[(df[column].notnull()) & (df['Gender'] == 'M')][column].count()
            female_percentage = df[(df[column].notnull()) & (df['Gender'] == 'F')][column].count()

            counts.append(int(round(male_percentage, 0)))
            labels.append(f'{column} - Male')
            parents.append(column)

            counts.append(int(round(female_percentage, 0)))
            labels.append(f'{column} - Female')
            parents.append(column)

        elif gender == 'Male':
            male_percentage = df[(df[column].notnull()) & (df['Gender'] == 'M')][column].count()

            counts.append(int(round(male_percentage, 0)))
            labels.append(f'{column} - Male')
            parents.append(column)

        elif gender == 'Female':
            female_percentage = df[(df[column].notnull()) & (df['Gender'] == 'F')][column].count()

            counts.append(int(round(female_percentage, 0)))
            labels.append(f'{column} - Female')
            parents.append(column)


        if comorbidity == 'All':
            yes_count = df[(df[column].notnull()) & (df['Comorbidities'] == True)][column].count()
            no_count = df[(df[column].notnull()) & (df['Comorbidities'] == False)][column].count()

            counts.append(int(round(yes_count, 0)))
            labels.append(f'{column} - Comorbidities True')
            parents.append(column)

            counts.append(int(round(no_count, 0)))
            labels.append(f'{column} - Comorbidities False')
            parents.append(column)

        elif comorbidity == 'Yes':
            yes_count = df[(df[column].notnull()) & (df['Comorbidities'] == True)][column].count()

            counts.append(int(round(yes_count, 0)))
            labels.append(f'{column} - Comorbidities True')
            parents.append(column)

        elif comorbidity == 'No':
            yes_count = df[(df[column].notnull()) & (df['Comorbidities'] == False)][column].count()

            counts.append(int(round(yes_count, 0)))
            labels.append(f'{column} - Comorbidities False')
            parents.append(column)


    fig = go.Figure(go.Treemap(
        labels=labels,
        parents=parents,
        values=counts,
        marker=dict(
            line=dict(width=2, color='#FFFFFF')
        )
    ))

    fig.update_layout(
        margin=dict(t=50, b=0, r=0, l=0),
    )

    return fig

In [7]:
stroke_data = df_stroke.copy()
stroke_data['Hospital Stay Duration (Days)'] = (stroke_data['Discharge Time'] - stroke_data['Admission Time']).dt.days
stroke_data['ICU Stay Duration (Days)'] = (stroke_data['ICU Checkout Time'] - stroke_data['ICU Arrival Time']).dt.days

stroke_data['ICU Stay Duration (Days)'].fillna(0, inplace=True)

bins = [0, 18, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91+']
stroke_data['Age Group'] = pd.cut(stroke_data['Age'], bins=bins, labels=labels, right=False)

light_template = go.layout.Template()
light_template.layout = go.Layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(color='black')
)


px.defaults.template = light_template


def plot_avg_duration(duration_type, comorbidities):

    y_column = f'{duration_type} (Days)'


    if comorbidities:
        filtered_data = stroke_data[stroke_data['Comorbidities'] == 1]
    else:
        filtered_data = stroke_data

    grouped_data = filtered_data.groupby('Age Group')[y_column].mean().reset_index()


    fig = px.bar(grouped_data, x='Age Group', y=y_column,
                 title=f'<b>Average {duration_type} Duration by Age Group</b>',
                 labels={y_column: 'Average Duration (Days)', 'Age Group': 'Age Group'},
                 color='Age Group')
    return dcc.Graph(figure=fig)

bins = [0, 18, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
stroke_data['Age Group'] = pd.cut(stroke_data['Age'], bins=bins, labels=labels, right=False)
stroke_data['Gender'] = stroke_data['Gender'].replace({'M': 'Male', 'F': 'Female'})
stroke_data['Comorbidities'] = stroke_data['Comorbidities'].apply(lambda x: "with Comorbidities" if x else "without Comorbidities")
stroke_data['Description'] = stroke_data['Gender'] + " " + stroke_data['Comorbidities']


age_gender_comorbidity_distribution = stroke_data.groupby(['Age Group', 'Gender', 'Comorbidities']).size().reset_index(name='Count')
age_gender_comorbidity_distribution['Description'] = age_gender_comorbidity_distribution['Gender'] + " " + age_gender_comorbidity_distribution['Comorbidities']

def plot_age_gender_comorbidity_distribution(selected_descriptions):
    color_map = {
        'Male with Comorbidities': 'rgba(146, 233, 249, 0.8)',
        'Male without Comorbidities': 'rgba(44, 195, 223, 0.8)',
        'Female with Comorbidities': 'rgba(254, 98, 110, 0.8)',
        'Female without Comorbidities': 'rgba(234, 26, 63, 0.8)'
    }


    fig = go.Figure()

    for selected_description in selected_descriptions:
        filtered_data = age_gender_comorbidity_distribution[
            age_gender_comorbidity_distribution['Description'] == selected_description
        ]

        if not filtered_data.empty:
            fig.add_trace(go.Scatter(x=filtered_data['Age Group'], y=filtered_data['Count'],
                                     mode='lines+markers',
                                     name=selected_description,
                                     line_shape='spline',
                                     fill='tonexty',
                                     line=dict(color=color_map[selected_description], width=2),
                                     marker=dict(size=8),
                                     fillcolor=color_map[selected_description]))

    fig.update_layout(title='Age Group, Gender, and Comorbidity Distribution of Stroke Patients',
                      xaxis_title="Age Group",
                      yaxis_title="Number of Patients",
                      legend_title="Patient Type",
                      template="plotly_white",
                      hovermode="x unified")

    return fig

In [8]:
app = dash.Dash(__name__)
app.title = "COMP 6934 – Second Project (202287115, 202196528)"
app.layout = html.Div([
    html.H1("COMP 6934 – Second Project"),
    html.H1("Ontario Hospital Complex - Stroke Care Dashboard"),
    html.P("Submitted By: Mehadi Hassan (202287115) & Mohammad Shehabul Islam (202196528)"),
    html.H2("Emergency to Other Departments"),
    html.P("Time Range:"),
    dcc.Dropdown(
        id='time_range',
        options=[{'label': k, 'value': k} for k, v in time_list.items()],
        value=list(time_list.keys())[0],
    ),
    html.P("Departments:"),
    dcc.Checklist(
        id='dept_checklist',
        options=[
            {'label': 'Admitted to the Hospital', 'value': 'admission_time'},
            {'label': 'CT Scan', 'value': 'ct_scan_time'},
            {'label': 'TPA', 'value': 'tpa_time'},
            {'label': 'Neurology Ward Arrival', 'value': 'neurologist_ward_time'},
            {'label': 'Neurologist Visit', 'value': 'neurologist_visit_time'},
        ],
        value=['ct_scan_time', 'tpa_time']
    ),
    html.P("Gender:"),
    dcc.Dropdown(
        id='gender_dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Male', 'value': 'Male'},
            {'label': 'Female', 'value': 'Female'}
        ],
        value='All',
    ),
    html.P("Comorbidity:"),
    dcc.Dropdown(
        id='comorbidity_dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Yes', 'value': 'Yes'},
            {'label': 'No', 'value': 'No'}
        ],
        value='All',
    ),
    dcc.Graph(id='dept_time_graph'),
    html.H2("Patients Requiring Extra Care"),
    html.P("Comorbidity:"),
    dcc.Dropdown(
        id='comorbidity_dropdown_for_extra',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Yes', 'value': 'Yes'},
            {'label': 'No', 'value': 'No'}
        ],
        value='All',
    ),
    html.P("Gender:"),
    dcc.Dropdown(
        id='gender_dropdown_for_extra',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Male', 'value': 'Male'},
            {'label': 'Female', 'value': 'Female'}
        ],
        value='All',
    ),
    html.P("Departments:"),
    dcc.Checklist(
        id='special_dept_checklist',
        options=[
            {'label': 'TPA', 'value': 'TPA Time'},
            {'label': 'ICU ', 'value': 'ICU Arrival Time'},
            {'label': 'Occupational Therapist Visit', 'value': 'Occupational Therapist Visit'},
            {'label': 'Speech Pathologist Visit', 'value': 'Speech Pathologist Visit'},
            {'label': 'Physiotherapist Visit', 'value': 'Physiotherapist Visit'},
            {'label': 'Dietitian Visit', 'value': 'Dietitian Visit'},
            {'label': 'Social Worker Visit', 'value': 'Social Worker Visit'},
            {'label': 'Cardiologist Visit', 'value': 'Cardiologist Visit'}
        ],
        value=['TPA Time', 'ICU Arrival Time', 'Occupational Therapist Visit', 'Speech Pathologist Visit']
    ),
    dcc.Graph(id='dept_extra_graph'),
    html.Div([
        html.H2("Duration of Stay of Different Age Groups"),
        html.Label('Select Duration:'),
        dcc.Dropdown(
            id='duration-type',
            options=[
                {'label': 'Hospital Stay Duration', 'value': 'Hospital Stay Duration'},
                {'label': 'ICU Stay Duration', 'value': 'ICU Stay Duration'}
            ],
            value='Hospital Stay Duration'
        ),
        html.Label('Comorbidities:'),
        dcc.Checklist(
            id='comorbidities',
            options=[
                {'label': 'Yes', 'value': True},
            ],
            value=[]
        ),
        html.Div(id='plot-output1')
    ], className="Hospital Ontario"),
    html.Div([
        html.H2("Age Group, Gender, and Comorbidity Distribution of Stroke Patients"),
        dcc.Checklist(
        id='description-checklist',
        options=[{'label': desc, 'value': desc} for desc in age_gender_comorbidity_distribution['Description'].unique()],
        value=[desc for desc in age_gender_comorbidity_distribution['Description'].unique()],
        labelStyle={'display': 'block'}
    ),
    dcc.Graph(id='age-gender-comorbidity-plot')
    ], className="Hospital Ontario")
], style={'font-family': 'Arial'})


@app.callback(
    Output('dept_time_graph', 'figure'),
    [
        Input('time_range', 'value'),
        Input('dept_checklist', 'value'),
        Input('gender_dropdown', 'value'),
        Input('comorbidity_dropdown', 'value')
    ]
)
def update_emergency_to_other_dept_graph(time_range, dept_checklist, gender, comorbidity):
    return emergency_to_other_dept_time(
        time_range,
        'admission_time' in dept_checklist,
        'ct_scan_time' in dept_checklist,
        'tpa_time' in dept_checklist,
        'neurologist_visit_time' in dept_checklist,
        'neurologist_ward_time' in dept_checklist,
        comorbidity,
        gender
    )


@app.callback(
    Output('dept_extra_graph', 'figure'),
    [
        Input('special_dept_checklist', 'value'),
        Input('gender_dropdown_for_extra', 'value'),
        Input('comorbidity_dropdown_for_extra', 'value'),
    ]
)
def update_extra_dept_care(special_dept_checklist, gender_dropdown_for_extra, comorbidity_dropdown_for_extra):
    return extra_dept_care(special_dept_checklist, gender_dropdown_for_extra, comorbidity_dropdown_for_extra)


@app.callback(
    Output('plot-output1', 'children'),
    [Input('duration-type', 'value'),
     Input('comorbidities', 'value')]
)
def update_plot(duration_type, comorbidities):
    return plot_avg_duration(duration_type, comorbidities)


@app.callback(
    Output('age-gender-comorbidity-plot', 'figure'),
    [Input('description-checklist', 'value')]
)
def update_age_gender_comorbidity_distribution(selected_description):
    return plot_age_gender_comorbidity_distribution(selected_description)

if __name__ == '__main__':
    app.run_server(debug=True)

### Acknowledgement


This assignment was completed by following the lecture materials provided in the course and the library documentations of Plotly and Plotly Dash. In addition to that to setup the dashboard this external resource was used - https://towardsdatascience.com/dash-for-beginners-create-interactive-python-dashboards-338bfcb6ffa4

N.B: Only the essential cells for the assignment has been kept and I have removed all the cells where I did initial analysis on data such as `info`, `describe`, `head`, `tail` and so on.
